# Modèle proie-prédateur dérivé de l'équation d'agrégation avec trois espèces

Partant du modèle proie-prédateur donné dans [[1]](https://arxiv.org/pdf/1912.06423.pdf), on peut imaginer un modèle pour trois espèces, et même pour $N\geq 3$ espèces. En jouant sur les potentiels d'interaction, les signes des interactions et les vitesses des différentes espèces, le cas $N=3$ donne déjà beaucoup de possibilités :

$$
\begin{cases}
\partial_t\rho_1 + \partial_x (\rho_1(W_1' * \rho_1\pm K'*\rho_2\pm K'*\rho_3)) &= 0, \\
\partial_t\rho_2 + \partial_x (\rho_2(W_2' * \rho_2\pm \beta_2 K'*\rho_1\pm \beta_2 K'*\rho_3)) &= 0, \\
\partial_t\rho_3 + \partial_x (\rho_3(W_3' * \rho_3\pm\beta_3 K'*\rho_1\pm\beta_3 K'*\rho_2)) &= 0, \\
\end{cases}
$$


où $\beta_2,\beta_3\in[0,1]$ sont les vitesses des espèces 2 et 3, $W_i$ est le potentiel d'interaction de l'espèce $i$ et $K$ est le potentiel d'interaction Interespèce.

Fixer les signes dans le système qui précède donne lieu à $2^6=64$ possibilités. Pour bien faire, il faudrait déduire les situations symétiques. Quoi qu'il en soit, on a une grande diversité de scénarios.

On peut aussi envisager un potentiel interespèce différent pour chaque couple d'espèces (ce qui n'est pas le cas dans le système écrit ici).

## Sommaire :
* [Paramètres/fonctions du problème](#t1)
* [Implémentation du schéma](#t2)

In [1]:
# Importations :
import numpy as np
import matplotlib.pyplot as plt
import os

### Définition des paramètres et fonctions du problème <a class="anchor" id="t1"></a>

In [2]:
# Potentiel d'interaction W1 et sa dérivée :
W1 = lambda x : np.abs(x) # Potentiel d'interaction
dxW1 = lambda x : np.sign(x) # Dérivée de W1

# Potentiel d'interaction W2 et sa dérivée :
W2   = lambda x : np.abs(x) # Potentiel d'interaction
dxW2 = lambda x : np.sign(x) # Dérivée de W2

# Potentiel d'interaction W2 et sa dérivée :
W3   = lambda x : np.abs(x) # Potentiel d'interaction
dxW3 = lambda x : np.sign(x) # Dérivée de W3

# Potentiel d'interaction K et sa dérivée :
K   = lambda x : np.abs(x) # Potentiel d'interaction
dxK = lambda x : np.sign(x) # Dérivée de K

# Fonctions pour initialiser la solution :
rho1_ini = lambda x : 1.*(x>-0.99)*(x<-0.98)
rho2_ini = lambda x : 1.*(x>-0.05)*(x<0.05)
rho3_ini = lambda x : 1.*(x>0.3)*(x<0.5)

# Définition de la grille spatiale :
xmin = -1 # Borne de gauche du domaine
xmax = 1 # Borne de droite du domaine
Nx = 1000 # Nombre de points de la discrétisation en espace
grid = np.linspace(xmin, xmax, Nx) # Grille
Dx = grid[1]-grid[0] # Pas d'espace

# Discrétisation en temps :
Nt = 4000 # Nombre de points de la discrétisation en temps
tf = 2 # Temps final
T = np.linspace(0, tf, Nt) # Temps discrets
Dt = T[1]-T[0] # Pas de temps

alpha2 = .7 # Vitesse des proies 2
alpha3 = .3 # Vitesse des proies 3

### Implémentation du schéma <a class="anchor" id="t2"></a>

In [3]:
# Création du fichier pour la sauvegarde des figures :
if not os.path.exists('./figures5'):
    os.makedirs('./figures5')

# Matrices pour contenir les potentiels :
W1p = dxW1(np.tile(grid.reshape((Nx, 1)), (1, Nx)) - grid)
W2p = dxW2(np.tile(grid.reshape((Nx, 1)), (1, Nx)) - grid)
W3p = dxW3(np.tile(grid.reshape((Nx, 1)), (1, Nx)) - grid)
Kp  = dxK(np.tile(grid.reshape((Nx, 1)), (1, Nx)) - grid)

# Initialisation rho1 :
rho1  = rho1_ini(grid)
rho1 /= sum(rho1) # Normalisation

# Initialisation rho2 :
rho2  = rho2_ini(grid)
rho2 /= sum(rho2) # Normalisation

# Initialisation rho3 :
rho3  = rho3_ini(grid)
rho3 /= sum(rho3) # Normalisation

# Plot de la condition initiale :
#plt.plot(grid, rho1, label=r"$\rho_1$")
#plt.plot(grid, rho2, label=r"$\rho_2$")
#plt.plot(grid, rho3, label=r"$\rho_2$")
#plt.legend()
#plt.show()

# Variables pour vitesses macroscopiques & variables intermédiaires :

a1 = np.zeros(Nx) # Pour la vitesse macroscopique a1
rho1_dp = np.zeros(Nx) # Pour contenir un vecteur "avec décalage"
rho1_gm = np.zeros(Nx) # Pour contenir un vecteur "avec décalage"

a2 = np.zeros(Nx) # Pour la vitesse macroscopique a2
rho2_dp = np.zeros(Nx) # Pour contenir un vecteur "avec décalage"
rho2_gm = np.zeros(Nx) # Pour contenir un vecteur "avec décalage"

a3 = np.zeros(Nx) # Pour la vitesse macroscopique a3
rho3_dp = np.zeros(Nx) # Pour contenir un vecteur "avec décalage"
rho3_gm = np.zeros(Nx) # Pour contenir un vecteur "avec décalage"

# Boucle de résolution :
cpt = 0 # Nombre de sauvegardes
for n,t in enumerate(T):

    # Calcul des vitesses macroscopiques a1, a2 et a3 :
    # (Avec produit matriciel)
    a1 = - (np.dot(W1p, rho1) + np.dot(Kp, rho2) + np.dot(Kp, rho3))
    a2 = - (np.dot(W2p, rho2) - alpha2 * np.dot(Kp, rho1) + alpha2 * np.dot(Kp, rho3))
    a3 = - (np.dot(W3p, rho3) - alpha3 * np.dot(Kp, rho2) - alpha3 * np.dot(Kp, rho1))
    
    # Mise à jour de rho1 :
    a1p = a1*(a1>0)  # Partie positive de a
    a1m = -a1*(a1<0) # Partie négative de a
    rho1_dp[1:]  = a1p[:-1]*rho1[:-1]
    rho1_gm[:-1] = a1m[1:]*rho1[1:]
    rho1 -= Dt/Dx*(a1p*rho1+a1m*rho1-rho1_dp-rho1_gm)
    
    # Mise à jour de rho2 :
    a2p = a2*(a2>0)  # Partie positive de a
    a2m = -a2*(a2<0) # Partie négative de a
    rho2_dp[1:]  = a2p[:-1]*rho2[:-1]
    rho2_gm[:-1] = a2m[1:]*rho2[1:]
    rho2 -= Dt/Dx*(a2p*rho2+a2m*rho2-rho2_dp-rho2_gm)
    
    # Mise à jour de rho3 :
    a3p = a3*(a3>0)  # Partie positive de a
    a3m = -a3*(a3<0) # Partie négative de a
    rho3_dp[1:]  = a3p[:-1]*rho3[:-1]
    rho3_gm[:-1] = a3m[1:]*rho3[1:]
    rho3 -= Dt/Dx*(a3p*rho3+a3m*rho3-rho3_dp-rho3_gm)
    
    # Pour un affichage de la solution à intervalles réguliers :
    if (not n%100):
        cpt+= 1
        plt.plot(grid, rho1, label=r"prédateurs $\rho_1$")
        plt.plot(grid, rho2, label=r"proies $\rho_2$")
        plt.plot(grid, rho3, label=r"proies $\rho_3$")
        plt.ylim([0, 1.1])
        plt.title("t = "+str(round(t,2)))
        plt.legend()
        plt.savefig("./figures5/etape"+str(cpt)+".pdf", facecolor='white', transparent=False)
        plt.clf()

<Figure size 432x288 with 0 Axes>